## Agent와 Multi-turn 대화하기

에이전트는 상태를 저장하지 않으며 대화 간에 내부적으로 어떠한 상태도 유지하지 않습니다. 에이전트와 여러 차례에 걸친 대화를 하려면 대화 상태를 저장할 객체를 생성하고, 이 객체를 에이전트 실행 시 전달해야 합니다. 대화 상태 객체를 생성하려면 에이전트 인스턴스에서 get_new_thread() 메서드를 호출합니다.

#### 에이전트 생성 및 쓰레드 생성

In [ ]:
import asyncio
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

# 새로운 에이전트 생성
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="You are good at telling jokes who responds in korean.",
    name="Joker"
)

# 새로운 쓰레드 생성
thread = agent.get_new_thread()


#### 다중 턴 대화로 에이전트 실행

In [ ]:
async def main():
    result1 = await agent.run(
        "Tell me a joke about a pirate.", 
        thread=thread)
    print("결과 1: " + result1.text)

    result2 = await agent.run(
        "Now add some emojis to the joke and tell it in the voice of a pirate's parrot.", 
        thread=thread)
    print("결과 2: " + result2.text)

await main()

> **중요 정보**
>
>에이전트에서 사용하는 서비스 유형에 따라 대화 기록 저장 방식이 결정됩니다. 예를 들어, 이 예제처럼 Chat Completion 서비스를 사용하는 경우 대화 기록은 AgentThread 개체에 저장되고 호출될 때마다 서비스로 전송됩니다. 반면 Azure AI Agent 서비스를 사용하는 경우에는 대화 기록이 Azure AI Agent 서비스에 저장되고 호출될 때마다 대화에 대한 참조만 서비스로 전송됩니다.

#### 단일 Agent가 멀티 쓰레드(대화)를 처리

여러 개의 AgentThread 객체를 생성하여 동일한 에이전트 인스턴스와 여러 개의 독립적인 대화를 진행할 수 있습니다. 이러한 스레드를 사용하여 각 대화에 대한 별도의 상태를 관리할 수 있습니다. 에이전트는 내부적으로 상태를 유지하지 않으므로 각 대화는 완전히 독립적입니다.

In [ ]:
async def main():
    thread1 = agent.get_new_thread()
    thread2 = agent.get_new_thread()

    result1 = await agent.run("Tell me a joke about a pirate.", thread=thread1)
    print("결과 1: " + result1.text)

    result2 = await agent.run("Tell me a joke about a robot.", thread=thread2)
    print("결과 2: " + result2.text)

    result3 = await agent.run("Now add some emojis to the joke and tell it in the voice of a pirate's parrot.", thread=thread1)
    print("결과 1-1: " + result3.text)

    result4 = await agent.run("Now add some emojis to the joke and tell it in the voice of a robot.", thread=thread2)
    print("결과 2-1: " + result4.text)

await main()